In [13]:
import numpy as np
import pandas as pd
import os
import matplotlib
%matplotlib inline
from matplotlib import pyplot as plt
import time

In [25]:

#file_name = "../nhussain_scripts/InvModShiftVals/10129857-10_feats.txt"
file_list = os.listdir("../../Training/simulation/primary/")[:1000000]
file_list_lc = [x for x in file_list if x[-2:] is "lc"]
print file_list[1:5]#[-3:]

print len(file_list_lc)
#star_list = [x.split("-")[0] for x in file_list]
#print len(np.unique(star_list))

['10000198-1_prim.txt', '10000198-2_prim.txt', '10000198-3_prim.txt', '10000198-4_prim.txt']
0


In [26]:
#10000198-0_prim_ltf.blsanal
#10000198-0_prim_ltf.lc
prim_ltf = np.loadtxt("../../Training/simulation/primary//prim_ltf.txt", dtype = str)[:20000]
print prim_ltf                     

['10000198-0_prim_ltf.lc' '10000198-10_prim_ltf.lc'
 '10000198-11_prim_ltf.lc' ..., '10848553-8_prim_ltf.lc'
 '10848553-9_prim_ltf.lc' '10849082-0_prim_ltf.lc']


In [27]:
koi_non_prim = np.zeros((20000,35))


In [ ]:
for QQ, name in enumerate(prim_ltf[100:20000]):#[20000:21000]):
    first_lc = np.loadtxt("../../Training/simulation/primary/" + name)
    first_bls = np.loadtxt("../../Training/simulation/primary/"+name[:-2]+"blsanal")[0, 1:]
    #break

    #fold it in phase space
    phase = ((first_lc[:,0] -first_bls[1])/first_bls[0])%1
    depth = first_bls[5]
    transit_duration = first_bls[6]
    ingress_duration = first_bls[7]*first_bls[6]*1.5


    #find white noise, and remove primary signal
    white_noise = np.std(first_lc[:,1][(phase>transit_duration/2) & (phase<1-transit_duration/2)])
    mean_out_transit_flux = np.mean(first_lc[:,1][(phase>transit_duration/2) & (phase<1-transit_duration/2)])
    #remove in transit portion

    cleaned_vals = np.empty(shape=[0,3])
    #remove transit signal
    slope = depth /ingress_duration


    for i, vals in enumerate(first_lc):
        vals =vals.reshape(1,3)
        if phase[i] <transit_duration-ingress_duration or phase[i]>1-transit_duration+ingress_duration:
             vals[0][1] = vals[0][1]-depth
        elif ((phase[i] > transit_duration-ingress_duration  and phase[i]<transit_duration)) :
             vals[0][1] -= (ingress_duration - phase[i]) *slope

        elif ((phase[i]>1-transit_duration) and (phase[i]<1-transit_duration+ingress_duration)):
             vals[0][1] -= (ingress_duration - (1 - phase[i])) *slope

        cleaned_vals = np.append(cleaned_vals, vals, axis = 0)



    #using transit width, search through phase space to find secondary and tertiary signals
    comb = zip(phase, cleaned_vals[:,1])
    comb.sort()
    phase_time, total_flux = zip(*comb)
    total_flux = np.asarray(total_flux).reshape((len(total_flux),1))

    
    mod_sig_sec_dv = 0
    mod_sig_pos_dv = 0
    sec_loc = 1
    phase_time = np.asarray(phase_time)

    cut_phase =  phase_time#[(phase_time > transit_duration) & (phase_time<(1-transit_duration))]
    cut_flux = total_flux#[(phase_time > transit_duration) & (phase_time<(1-transit_duration))]

    for i, vals in enumerate(cut_phase):
        section = cut_flux[(cut_phase>cut_phase[i]) & (cut_phase<cut_phase[i]+transit_duration*2)]
        depth_two = np.mean(section) - mean_out_transit_flux
        if depth_two == max( mod_sig_sec_dv, depth_two):
            sec_loc = phase_time[i]
        mod_sig_sec_dv = max(mod_sig_sec_dv, depth_two)
        mod_sig_pos_dv = min(mod_sig_pos_dv, depth_two)




    for i, vals in enumerate(cut_phase):
        if (vals< sec_loc+transit_duration) and (vals>sec_loc-transit_duration):
            cut_flux[i] -= mod_sig_sec_dv

    mod_sig_ter_dv = 0
    for i, vals in enumerate(cut_phase):
        section = cut_flux[(cut_phase>cut_phase[i]) & (cut_phase<cut_phase[i]+transit_duration*2)]
        sig = np.mean(section) - mean_out_transit_flux
        mod_sig_ter_dv = max(mod_sig_sec_dv, sig)

    mod_sig_sec_dv = mod_sig_sec_dv - mean_out_transit_flux


    #red noise
    #red noise^2 = white noise /npoiint + red noise
    #inverse relationship
    #try with three timescales, one, two, and threee t durations
    #look at portions outside of transit?
    red_noise_1 = []
    red_noise_2 = []
    red_noise_3 = []

    red_noise_1_p = []
    red_noise_2_p = []
    red_noise_3_p = []

    for i, vals in enumerate(cut_phase[:-1]):
        section1 = cut_flux[(cut_phase>cut_phase[i]) & (cut_phase<cut_phase[i]+transit_duration)]
        section2 = cut_flux[(cut_phase>cut_phase[i]) & (cut_phase<cut_phase[i]+transit_duration*2)]
        section3 = cut_flux[(cut_phase>cut_phase[i]) & (cut_phase<cut_phase[i]+transit_duration*3)]
        red_noise_1_p.append(section1.shape[0])
        red_noise_2_p.append(section2.shape[0])
        red_noise_3_p.append(section3.shape[0])
        red_noise_1.append(np.var(section1))
        red_noise_2.append(np.var(section2))
        red_noise_3.append(np.var(section3))
    
    N_p_1 = np.median(red_noise_1_p)
    N_p_2 = np.median(red_noise_2_p)
    N_p_3 = np.median(red_noise_3_p)

    white_noise_2 = np.abs((np.mean(red_noise_1) - np.mean(red_noise_2))* (N_p_1 * N_p_2) / (N_p_1+N_p_2))

    

    red_noise = np.sqrt(np.abs(np.mean(red_noise_1) - white_noise_2/N_p_1))

    from scipy.special import erfcinv

    period = first_bls[0]
    n_trans = first_bls[14]

 

    mod_sig_pri_dv = depth
    mod_fred_dv = red_noise / white_noise # ratio between the two
    mod_fa1_dv = np.sqrt(2) * erfcinv(transit_duration/(period* n_trans))
    mod_fa2_dv = np.sqrt(2) * erfcinv(transit_duration/(period))

    modshiftval1_dv = mod_sig_pri_dv/mod_fred_dv - mod_fa1_dv
    modshiftval2_dv = mod_sig_pri_dv - mod_sig_ter_dv-mod_fa2_dv
    modshiftval3_dv = mod_sig_pri_dv - mod_sig_pos_dv - mod_fa2_dv
    modshiftval4_dv = mod_sig_sec_dv / mod_fred_dv - mod_fa1_dv
    modshiftval5_dv = mod_sig_sec_dv - mod_sig_ter_dv - mod_fa2_dv
    modshiftval6_dv = mod_sig_sec_dv-mod_sig_pos_dv - mod_fa2_dv


    vals = [mod_sig_pri_dv, mod_sig_sec_dv, mod_sig_ter_dv, mod_sig_pos_dv, mod_fa1_dv, mod_fa2_dv, mod_fred_dv, modshiftval1_dv, modshiftval2_dv, modshiftval3_dv, modshiftval4_dv, modshiftval5_dv, modshiftval6_dv, white_noise, red_noise]

    total_Vals = np.concatenate([vals, first_bls])

    head_txt = """mod_sig_pri_dv mod_sig_sec_dv mod_sig_ter_dv mod_sig_pos_dv mod_fa1_dv mod_fa2_dv mod_fred_dv modshiftval1_dv modshiftval2_dv modshiftval3_dv modshiftval4_dv modshiftval5_dv modshiftval6_dv white_noise red_noise BLS_Period_1_0 BLS_Tc_1_0 BLS_SN_1_0 BLS_SR_1_0 BLS_SDE_1_0 BLS_Depth_1_0 BLS_Qtran_1_0 BLS_Qingress_1_0 BLS_OOTmag_1_0 BLS_i1_1_0 BLS_i2_1_0 BLS_deltaChi2_1_0 BLS_fraconenight_1_0 BLS_Npointsintransit_1_0 BLS_Ntransits_1_0 BLS_Npointsbeforetransit_1_0 BLS_Npointsaftertransit_1_0 BLS_Rednoise_1_0 BLS_Whitenoise_1_0 BLS_SignaltoPinknoise_1_0 """
    np.savetxt("../KOInotPrimaryModSHiftVals/" + name[:-8] +"_feats_fixed.txt", total_Vals, header=head_txt)
    if QQ %10 == 9:
        print QQ
    #break
    #break

9
19
29
39


In [22]:
#read values from file and put it together in csv file
#10186761-12_pri_feats_fixed.txt
total_values = np.zeros((20000,35))
for i, name in enumerate(prim_ltf[:20000]):
    print name
    data =np.loadtxt("../KOInotPrimaryModSHiftVals/"+name[:-8]+"_feats_fixed.txt")
    total_values[i, :] = data
    

10000198-0_prim_ltf.lc
10000198-10_prim_ltf.lc
10000198-11_prim_ltf.lc
10000198-12_prim_ltf.lc
10000198-13_prim_ltf.lc
10000198-1_prim_ltf.lc
10000198-2_prim_ltf.lc
10000198-3_prim_ltf.lc
10000198-4_prim_ltf.lc
10000198-5_prim_ltf.lc
10000198-6_prim_ltf.lc
10000198-7_prim_ltf.lc
10000198-8_prim_ltf.lc
10000198-9_prim_ltf.lc
10002203-0_prim_ltf.lc
10002203-10_prim_ltf.lc
10002203-11_prim_ltf.lc
10002203-12_prim_ltf.lc
10002203-13_prim_ltf.lc
10002203-14_prim_ltf.lc
10002203-15_prim_ltf.lc
10002203-1_prim_ltf.lc
10002203-2_prim_ltf.lc
10002203-3_prim_ltf.lc
10002203-4_prim_ltf.lc
10002203-5_prim_ltf.lc
10002203-6_prim_ltf.lc
10002203-7_prim_ltf.lc
10002203-8_prim_ltf.lc
10002203-9_prim_ltf.lc
10002349-0_prim_ltf.lc
10002349-10_prim_ltf.lc
10002349-11_prim_ltf.lc
10002349-12_prim_ltf.lc
10002349-13_prim_ltf.lc
10002349-14_prim_ltf.lc
10002349-15_prim_ltf.lc
10002349-16_prim_ltf.lc
10002349-1_prim_ltf.lc
10002349-2_prim_ltf.lc
10002349-3_prim_ltf.lc
10002349-4_prim_ltf.lc
10002349-5_prim_l

In [23]:
# read all the KOI data and save features into one large csv flie
col_names = ["mod_sig_pri_dv", "mod_sig_sec_dv","mod_sig_ter_dv",
             "mod_sig_pos_dv", "mod_fa1_dv","mod_fa2_dv", "mod_fred_dv",
             "modshiftval1_dv", "modshiftval2_dv", "modshiftval3_dv",
             "modshiftval4_dv", "modshiftval5_dv", "modshiftval6_dv", 
             "white_noise", "red_noise", "BLS_Period_1_0", "BLS_Tc_1_0",
             "BLS_SN_1_0", "BLS_SR_1_0", "BLS_SDE_1_0", "BLS_Depth_1_0",
             "BLS_Qtran_1_0", "BLS_Qingress_1_0", "BLS_OOTmag_1_0",
             "BLS_i1_1_0", "BLS_i2_1_0", "BLS_deltaChi2_1_0", 
             "BLS_fraconenight_1_0", "BLS_Npointsintransit_1_0",
             "BLS_Ntransits_1_0", "BLS_Npointsbeforetransit_1_0", 
             "BLS_Npointsaftertransit_1_0", "BLS_Rednoise_1_0",
             "BLS_Whitenoise_1_0", "BLS_SignaltoPinknoise_1_0"]

print total_values[0, :]
#print type(total_values)
total_values_frame = pd.DataFrame(total_values, columns= col_names)


[  2.60000000e-04  -9.99981420e+00   1.72699435e-04  -1.67444764e-04
   3.02916625e+00   2.21187553e+00   1.00738658e+00  -3.02890815e+00
  -2.21178823e+00  -2.21144809e+00  -1.29556576e+01  -1.22118624e+01
  -1.22115223e+01   3.17143739e-04   3.19486346e-04   2.67244812e+00
   1.34800472e+02   4.62212000e+00   6.00000000e-05   4.68014000e+00
   2.60000000e-04   7.20900000e-02   1.90810000e-01   9.99999000e+00
   7.04940000e-01   7.77020000e-01  -0.00000000e+00   1.52110000e-01
   9.70000000e+01   1.10000000e+01   9.80000000e+01   9.00000000e+01
   7.00000000e-05   2.90000000e-04   7.08170000e+00]


In [24]:
total_values_frame.to_csv("../Data/KOI_non_prim_feat_files_20000_new_detrending.csv")
total_values_frame.shape

(20000, 35)